<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo3/M6U3_Taller_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=14reVO1X6LsjqJ3cFgoeHxxddZVGfZn3t" alt = "Encabezado MLDS" width = "100%">  </img>

# **Taller 3: Ciclo de vida de ciencia de datos**
---

En este notebook evaluaremos los conceptos aprendidos sobre el ciclo de vida de ciencia de datos. En especial, entrenaremos un modelo con la librería `xgboost` con su debida optimización de hiperparámetros.

Ejecute las siguientes celdas para conectarse a UNCode:

In [ ]:
!pip install rlxcrypt
!wget --no-cache -O session.pye -q https://raw.githubusercontent.com/JuezUN/INGInious/master/external%20libs/session.pye

In [ ]:
import rlxcrypt
import session

grader = session.LoginSequence("MAPEDDACML-GroupMLDS-6-2024-2@f8879e0a-fcd1-4b6b-a12f-31426dfcd762")

Comenzamos instalando las librerías y herramientas necesarias:

In [ ]:
!pip install mlflow
!pip install optuna optuna-dashboard mlflow xgboost
!pip install -U scikit-learn

Importamos las librerías necesarias:

In [ ]:
# Librerías de utilidad para manipulación y visualización de datos.
import os, mlflow, optuna
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from IPython import get_ipython
from IPython.display import display
plt.style.use("ggplot")

# Ignorar warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Versiones de las librerías usadas.
import sklearn
!python --version
print('MLflow', mlflow.__version__)
print('Optuna', optuna.__version__)
print('Scikit-learn', sklearn.__version__)

Esta actividad se realizó con las siguientes versiones:
*  Python 3.10.12
*  Scikit-learn 1.4.2
*  MLflow 2.12.1
*  Optuna 3.6.1

> **La tarea es incremental, por lo tanto es recomendable resolver los puntos en orden**

## **Carga de datos**
---

En este caso, utilizaremos un conjunto de datos sintético generado desde `sklearn`. Se trata de un *dataset* sintético utilizado comúnmente para tareas de clasificación binaria en aprendizaje automático. Este conjunto de datos consta de dos características continuas y una etiqueta binaria que indica a qué círculo pertenece cada punto.

Específicamente, la función `make_circles` crea un conjunto de puntos distribuidos uniformemente en dos círculos concéntricos, donde la distancia entre los dos círculos es ajustable. La distribución de puntos dentro de cada círculo se controla mediante el parámetro `noise`, que agrega ruido aleatorio a la posición de cada punto.

En general, este conjunto de datos se utiliza para evaluar la capacidad de los algoritmos de clasificación para separar clases no lineales en un espacio bidimensional. Debido a que los dos círculos se superponen, es imposible separar completamente las dos clases con una frontera de decisión lineal. Por lo tanto, se requieren técnicas más avanzadas, como la utilización de modelos no lineales, para clasificar adecuadamente los puntos en este conjunto de datos.

Vamos a generarlo:

In [ ]:
features, labels = make_circles(
    n_samples=1000,
    noise=0.1,
    factor=0.5,
    random_state=0
    )

Podemos visualizar el conjunto de datos:

In [ ]:
## KEEPOUTPUT
fig, ax = plt.subplots()
ax.scatter(features[:, 0], features[:, 1], c=labels, alpha=0.5, cmap="RdBu")
ax.set(xlabel="$x_1$", ylabel="$x_2$")
fig.show()

Adicionalmente, vamos a configurar el servidor de `mlflow`:

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

Utilizaremos `ngrok` para acceder al tablero de `mlflow`:

In [ ]:
!pip install pyngrok

Ahora debe agregar su token de `ngrok`:

In [ ]:
token = "" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Creamos un experimento:

In [ ]:
exp = mlflow.create_experiment(name="circles", artifact_location="mlruns")

Dividimos el conjunto de datos en entrenamiento y prueba:

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(
        features, labels, test_size=0.3, random_state=0
        )

## **1. Entrenamiento de Modelo**
---

En este punto deberá implementar una función que permita entrenar un modelo de `xgboost` dados los datos de entrenamiento y los hiperparámetros que exploráremos más adelante.

Para esto debe implementar la función `train_model` la cual toma como entrada las características y etiquetas de entrenamiento, la profundidad de los árboles, el número de estimadores, y la taza de aprendizaje. La función debe retornar el modelo entrenado.

**Parámetros**

- `features`: matriz de características de entrenamiento.
- `labels`: vector de etiquetas de entrenamiento.
- `max_depth`: profundidad máxima del árbol.
- `n_estimators`: número de estimadores.
- `learning_rate`: taza de aprendizaje.
- `random_state`: semilla de números aleatorios.

**Retorna**

- `model`: modelo de `xgboost` entrenado.

In [ ]:
# FUNCIÓN CALIFICADA train_model:
def train_model(
    features,
    labels,
    max_depth,
    n_estimators,
    learning_rate,
    random_state
    ):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = ...
    return model
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
model = train_model(
        features=features_train,
        labels=labels_train,
        max_depth=2,
        n_estimators=10,
        learning_rate=1e-4,
        random_state=0
        )
print(model.max_depth)
print(model.n_estimators)
print(model.learning_rate)

**Salida esperada**

En este caso debería obtener los hiperparámetros del modelo:

```python
❱ print(model.max_depth)
2

❱ print(model.n_estimators)
10

❱ print(model.learning_rate)
0.0001
```

In [ ]:
#TEST_CELL
model = train_model(
        features=features_train,
        labels=labels_train,
        max_depth=2,
        n_estimators=10,
        learning_rate=1e-4,
        random_state=0
        )
print(model.score(features_test, labels_test))

**Salida esperada**

En este caso debería obtener el accuracy del modelo:

```python
❱ print(model.score(features_test, labels_test))
0.48333333333333334
```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde que `XGBClassifier` funciona de una forma equivalente a `sklearn`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Valide que está usando los parámetros de la función y no las variables globales.
</details>

#### **Evaluar código**

In [ ]:
grader.run_test("Test 1_1", globals())

## **2. Evaluación del modelo**
---

En este punto debe implementar una función que permita calcular el `f1_score` sobre el conjunto de evaluación a partir de un modelo entrenado.

Para esto, debe implementar la función `eval_model`, la cual toma como entrada un modelo entrenado, las características y el vector de etiquetas de evaluación. Debe retornar el valor de la métrica.

**Parámetros**

- `model`: modelo entrenado.
- `features`: conjunto de datos de evaluación.
- `labels`: etiquetas de evaluación.

**Retorna**

- `score`: f1-score.

In [ ]:
# FUNCIÓN CALIFICADA eval_model:
def eval_model(
    model,
    features,
    labels,
    ):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    score = ...
    return score
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
model = train_model(
        features=features_train,
        labels=labels_train,
        max_depth=4,
        n_estimators=100,
        learning_rate=1e-3,
        random_state=0
        )
score = eval_model(model, features_test, labels_test)
print(score)

**Salida esperada**:

En este caso debería obtener la métrica de desempeño para el modelo de los hiperparámetros dados.

```python
❱ print(score)
0.0.9770491803278688
```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Para evaluar el f1-score puede usar la función `f1_score` de `sklearn`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Debe obtener las predicciones del modelo con el método `predict`.
</details>

#### **Evaluar código**

In [ ]:
grader.run_test("Test 2_1_1", globals())

In [ ]:
grader.run_test("Test 2_1_2", globals())

## **3. Ejecución en MLFlow**
---

Ahora, deberá crear una función que permita crear un **run** en `mlflow` para entrenar el modelo con sus correspondientes hiperparámetros bajo un experimento específico. Debe registrar el modelo, los hiperparámetros y la métrica del modelo que calcula en el punto anterior.

Para ello deberá implementar la función `mlflow_run`, la cual toma como entrada las características y etiquetas de entrenamiento, la profundidad máxima del modelo, el número de estimadores, la taza de aprendizaje y el experimento de `mlflow`. Debe retornar la ejecución y el valor de la métrica del modelo.

**Parámetros**

- `features_train`: matriz de características de entrenamiento.
- `labels_train`: vector de etiquetas de entrenamiento.
- `features_test`: matriz de características de evaluación.
- `labels_test`: vector de etiquetas de evaluación.
- `max_depth`: profundidad máxima del árbol.
- `n_estimators`: número de estimadores.
- `learning_rate`: taza de aprendizaje.
- `random_state`: semilla de números aleatorios.
- `exp`: experimento de `mlflow`.
- `run_name`: nombre a asignar a la ejecución.

**Retorna**

- `run`: ejecución de `mlflow`.
- `score`: valor de la métrica en la ejecución.

In [ ]:
# FUNCIÓN CALIFICADA mlflow_run:
def mlflow_run(
    features_train,
    labels_train,
    features_test,
    labels_test,
    max_depth,
    n_estimators,
    learning_rate,
    random_state,
    exp,
    run_name
    ):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    run = ...
    score = ...
    return run, score
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
run, score = mlflow_run(
        features_train=features_train,
        labels_train=labels_train,
        features_test=features_test,
        labels_test=labels_test,
        max_depth=4,
        n_estimators=100,
        learning_rate=1e-3,
        random_state=0,
        exp=exp,
        run_name="test_case"
        )
print(run.info.run_name)
print(os.listdir(run.info.artifact_uri))
print(score)

**Salida esperada**:

En este caso debería obtener los metadatos de la ejecución y el valor de la métrica:

```python
❱ print(run.info.run_name)
test_case

❱ print(os.listdir(run.info.artifact_uri))
['model']

❱ print(score)
0.9770491803278689
```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde usar de forma adecuada las particiones de entrenamiento y prueba con las funciones `train_model` y `eval_model` respectivamente.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Recuerde terminar la ejecución con la función `mlflow.end_run()`.
</details>

#### **Evaluar código**

In [ ]:
grader.run_test("Test 3_1_1", globals())

In [ ]:
grader.run_test("Test 3_1_2", globals())

## **4. Optimización de Hiperparámetros**
---

Ahora debe implementar la función objetivo para optimizar los hiper-parámetros con optuna. En específico debe variar los valores de la siguiente forma:

- `max_depth`: valor entero entre 2 y 10.
- `n_estimators`: valor entero entre 25 y 200.
- `learning_rate`: valor continuo entre 1e-6 y 1 (variaciones logarítmicas).

Todos los intentos deben estar registrados dentro de `mlflow`, para esto debe utilizar el experimento que está definido en la variable `exp`, como `run_name` debe utilizar el valor `"optuna"` y debe utilizar el valor 0 como `random_state`.

Debe implementar la función `objective` la cual toma como entrada un trial de `optuna` y debe retornar el valor de la métrica a maximizar.

**Parámetros**:

- `trial`: objeto `trial` de `optuna`.

**Retorna**:

- `score`: f1-score.

In [ ]:
# FUNCIÓN CALIFICADA objective:
def objective(trial):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    score = ...
    return score
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///hp.db",
    study_name="circles",
    )
study.optimize(func=objective, n_trials=30, n_jobs=1)

Si se dirige al dashboard de `mlflow`, deberá obtener varias ejecuciones bajo el nombre `optuna`. Puede filtrarlas todas al poner el filtro que se muestra en la imagen:

<img src="https://drive.google.com/uc?export=view&id=1j_7LYVWNADfVTW3YqUzhpcEHOaEPXXZ7" width="80%">

También debe seleccionar todos los runs con el nombre `optuna` y dar click en `compare`. Esto debe generar el siguiente resultado:

<img src="https://drive.google.com/uc?export=view&id=1CFnfmGdu0pgUbrGri-T8oNSafxQZAHSn" width="80%">

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Puede utilizar el método `suggest_int` de un `Trial` para generar un hiperparámetro de tipo entero.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Puede utilizar el método `suggest_float` de un `Trial` para generar un hiperparámetro continúo.
</details>

## **5. Versionado de Modelo**
---

Por último, en este punto deberá generar una versión del mejor modelo con el nombre `xgboost` versión 1. Posteriormente, debe implementar una función que permita cargar el modelo:

Para esto deberá implementar la función `load_model` la cual debe retornar el modelo versionado como `xgboost` versión 1:

**Parámetros**

La función no tiene parámetros de entrada.

**Retorna**

- `model`: modelo cargado con `mlflow`.

In [ ]:
# FUNCIÓN CALIFICADA load_model:
def load_model():
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = ...
    return model
    ### FIN DEL CÓDIGO ###

Use las siguientes celdas para probar su solución:

In [ ]:
#TEST_CELL
model = load_model()
y_pred = model.predict(features_test)
print(f1_score(labels_test, y_pred))

**Salida esperada**

En este caso debería obtener la métrica sobre el mejor modelo en el conjunto de evaluación.

```python
❱ print(f1_score(labels_test, y_pred))
0.9871794871794872
```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Recuerde versionar el modelo antes de cargarlo.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Puede ordenar las ejecuciones de `mlflow` de acuerdo a `score` y con esto seleccionar el mejor modelo.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Puede cargar un modelo versionado con la función `mlflow.pyfunc.load_model`.
</details>

#### **Evaluar código**

In [ ]:
grader.run_test("Test 5_1", globals())

# **Evaluación**

In [ ]:
grader.submit_task(globals())

# **Créditos**
---

* **Profesor:** [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/).

* **Asistentes docentes:** [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

* **Coordinador de virtualización:** [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*